## 🔀 MAF — Workflow 개요

> **AI 에이전트를 비즈니스 프로세스와 원활하게 통합하는 지능형 자동화 시스템을 구축하세요** ✨
>
> 📖 원본 링크: https://learn.microsoft.com/en-us/agent-framework/workflows/

---

### 🤔 워크플로우 vs 에이전트

에이전트와 워크플로우는 모두 여러 단계를 거쳐 목표를 달성하지만, **목적**과 **추상화 수준**이 다릅니다.

| 구분 | 🤖 에이전트 (Agent) | 🔄 워크플로우 (Workflow) |
|------|-------------------|----------------------|
| |![agent](./images/wf001.png)|![wf](./images/wf002.png)|
| **구동 방식** | LLM(대규모 언어 모델) | 미리 정의된 프로세스 |
| **실행 단계** | 동적 — 맥락과 도구에 따라 LLM이 결정 | 정적 — 명확하게 정의된 실행 경로 |
| **구성 요소** | 다양한 도구를 사용하는 단일 에이전트 | **여러 에이전트** + 사람의 개입 + 외부 시스템 |
| **제어 수준** | 유연하지만 예측 불가 | **세밀한 실행 경로 제어** |
| **적합한 상황** | 창의적·맥락 의존적 작업 | 복잡한 비즈니스 프로세스 자동화 |


### ⭐ 주요 특징 (Key Features)

MAF Workflows는 다음과 같은 강력한 기능을 제공합니다:

| 특징 | 설명 |
|------|------|
| 🔒 **타입 안전성** | 강력한 타입 검사로 컴포넌트 간 메시지의 정확한 전달을 보장하고, 포괄적인 유효성 검사로 런타임 오류를 방지 |
| 🌐 **유연한 제어 흐름** | 그래프 기반 아키텍처로 조건부 라우팅, 병렬 처리, 동적 실행 경로를 모두 지원 |
| 🔌 **외부 통합** | 외부 API와의 원활한 통합 및 사용자 개입 시나리오를 위한 내장 요청/응답 패턴 제공 |
| 💾 **체크포인트** | 워크플로우 상태를 저장하여 장시간 실행 프로세스의 복구 및 재개 가능 |
| 🤝 **다중 에이전트 오케스트레이션** | 순차적·동시적·핸드오프 방식으로 여러 AI 에이전트를 조율하는 내장 패턴 제공 |


### 🧩 핵심 개념 (Core Concepts)

워크플로우를 구성하는 **4가지 핵심 빌딩 블록**을 이해하세요:

#### ⚙️ 1. 실행기 (Executor)

워크플로 내의 **개별 처리 단위**입니다.

- AI 에이전트 또는 사용자 정의 로직 컴포넌트가 될 수 있습니다
- 입력 메시지를 수신 → 특정 작업 수행 → 출력 메시지 생성
- **2가지 구현 방법:**
  1. `@handler` 메서드를 포함하는 `Executor` 서브클래스
  2. `@executor` 데코레이터로 표시된 독립형 비동기 함수

```python
# 방법 1: 클래스 기반 Executor
class MyExecutor(Executor):
    @handler
    async def process(self, text: str, ctx: WorkflowContext[str]) -> None:
        await ctx.send_message(text.upper())

# 방법 2: 함수 기반 @executor
@executor(id="my_executor")
async def my_executor(text: str, ctx: WorkflowContext[Never, str]) -> None:
    await ctx.yield_output(text[::-1])
```

---

### 🔗 2. 엣지 (Edge)

실행기 간의 **연결**을 정의하여 메시지 흐름을 결정합니다.

- 메시지 내용에 따라 **라우팅을 제어하는 조건** 포함 가능
- 조건부·병렬 처리도 지원

---

### 📡 3. 이벤트 (Event)

워크플로우 실행에 대한 **관찰 가능성(Observability)** 을 제공합니다.(아래 표는 일부 이벤트만 표기)

| 이벤트 유형 | 발생 시점 |
|------------|---------|
| 🚀 `ExecutorInvokedEvent` | Executor가 처리를 시작할 때 |
| ✅ `ExecutorCompletedEvent` | Executor가 처리를 완료했을 때 |
| 🎯 `WorkflowOutputEvent` | 최종 워크플로우 결과가 생성될 때 |
| 🏁 `WorkflowStartedEvent` | 워크플로우가 시작될 때 |

---

### 🏗️ 4. 워크플로우 빌더 (WorkflowBuilder)

Executor와 Edge를 **방향 그래프**로 연결하고 실행을 관리합니다.


In [ ]:
# 🏗️ WorkflowBuilder — Fluent API 패턴 예시

# WorkflowBuilder는 직관적인 메서드 체이닝(fluent interface)으로
# 워크플로우를 선언적으로 구성할 수 있습니다.

workflow = (
    WorkflowBuilder()
    .add_edge(executor_a, executor_b)   # 🔗 Executor 간 연결 추가
    .add_edge(executor_b, executor_c)   # 🔗 체인 형태로 계속 연결
    .set_start_executor(executor_a)     # 🚪 시작 진입점 정의
    .build()                            # ✅ 워크플로우 객체 생성
)

# 스트리밍 실행 — 이벤트를 실시간으로 관찰
async for event in workflow.run_stream("입력 데이터"):
    print(f"Event: {event}")
    if isinstance(event, WorkflowOutputEvent):
        print(f"최종 결과: {event.data}")


### 🔧 WorkflowBuilder 메서드 요약

| 메서드 | 역할 |
|--------|------|
| 🔗 `.add_edge(src, dst)` | Executor 간 직접 연결 생성 |
| 🚪 `.set_start_executor(e)` | 워크플로우 진입점 정의 |
| ✅ `.build()` | 워크플로우 객체 생성 및 반환 |

---

## 🗺️ 다음 단계

| 번호 | 노트북 | 내용 |
|------|--------|------|
| 21 | `21.SimpleSequentialWorkflow.ipynb` | 간단한 순차 워크플로우 구현 |
| 22 | `22.Agents-In-Workflow.ipynb` | 워크플로우 내 AI 에이전트 통합 |
